In [96]:
import pandas as pd
import re
import os

"""

refseq - ncbi-ftp/gene/DATA/gene2refseq.gz
pubmed - ncbi-ftp/gene/DATA/gene2pubmed.gz
ensembl - ncbi-ftp/gene/DATA/gene2ensembl.gz
human - ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz
mart - useast.ensembl.org/index.html
neighbors - ncbi-ftp/gene/DATA/gene_neighbors.gz
orthologs - ncbi-ftp/gene/DATA/gene_orthologs.gz

"""

'\n\nrefseq - ncbi-ftp/gene/DATA/gene2refseq.gz\npubmed - ncbi-ftp/gene/DATA/gene2pubmed.gz\nensembl - ncbi-ftp/gene/DATA/gene2ensembl.gz\nhuman - ftp://ftp.ncbi.nlm.nih.gov/gene/DATA/GENE_INFO/Mammalia/Homo_sapiens.gene_info.gz\nmart - useast.ensembl.org/index.html\nneighbors - ncbi-ftp/gene/DATA/gene_neighbors.gz\northologs - ncbi-ftp/gene/DATA/gene_orthologs.gz\n\n'

In [21]:
refseq = pd.read_table('gene2refseq')
refseq

C:\Users\Brendan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,6,8,9,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#tax_id,GeneID,status,RNA_nucleotide_accession.version,RNA_nucleotide_gi,protein_accession.version,protein_gi,genomic_nucleotide_accession.version,genomic_nucleotide_gi,start_position_on_the_genomic_accession,end_position_on_the_genomic_accession,orientation,assembly,mature_peptide_accession.version,mature_peptide_gi,Symbol
0,9,1246500,PROVISIONAL,-,-,NP_047184.1,10954455,NC_001911.1,10954454,348,1190,-,-,-,-,repA1
1,9,1246501,PROVISIONAL,-,-,NP_047186.1,10954457,NC_001911.1,10954454,2157,2912,+,-,-,-,repA2
2,9,1246502,VALIDATED,-,-,NP_047187.1,10954458,NC_001911.1,10954454,3040,4590,+,-,-,-,leuA
3,9,1246503,PROVISIONAL,-,-,NP_047188.1,10954459,NC_001911.1,10954454,4623,5714,+,-,-,-,leuB
4,9,1246504,VALIDATED,-,-,NP_047189.1,10954460,NC_001911.1,10954454,5717,7117,+,-,-,-,leuC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40141439,2749911,36959328,NaN,-,-,-,-,NC_038002.1,1418587958,119755,119828,-,-,-,-,trnP-GGG
40141440,2749911,36959329,PROVISIONAL,-,-,YP_009496746.1,1418587959,NC_038002.1,1418587958,116,2374,+,-,-,-,psaA
40141441,2749911,36959330,PROVISIONAL,-,-,YP_009496877.1,1418588090,NC_038002.1,1418587958,120098,120337,-,-,-,-,acpP
40141442,2749911,36959331,PROVISIONAL,-,-,YP_009496787.1,1418588000,NC_038002.1,1418587958,40018,40134,+,-,-,-,psbI


In [22]:
pubmed = pd.read_table('gene2pubmed')
pubmed

,#tax_id,GeneID,PubMed_ID
0,9,1246500,9873079
1,9,1246501,9873079
2,9,1246502,9873079
3,9,1246503,9873079
4,9,1246504,9873079
...,...,...,...
12757291,2747817,55898830,19273270
12757292,2747817,55898833,22037399
12757293,2747817,55898833,23651326
12757294,2747817,55898833,26712937


In [23]:
ensembl = pd.read_table('gene2ensembl')
ensembl

,#tax_id,GeneID,Ensembl_gene_identifier,RNA_nucleotide_accession.version,Ensembl_rna_identifier,protein_accession.version,Ensembl_protein_identifier
0,7227,30970,FBgn0040373,NM_001297803.1,FBtr0340207,NP_001284732.1,FBpp0309182
1,7227,30970,FBgn0040373,NM_130477.4,FBtr0070108,NP_569833.1,FBpp0070103
2,7227,30970,FBgn0040373,NM_166834.2,FBtr0070107,NP_726658.1,FBpp0070102
3,7227,30971,FBgn0040372,NM_001272159.1,FBtr0332992,NP_001259088.1,FBpp0305207
4,7227,30971,FBgn0040372,NM_130478.3,FBtr0070063,NP_569834.1,FBpp0070062
...,...,...,...,...,...,...,...
3402972,2587831,115271344,ENSTMTG00000012187,XM_029913242.1,ENSTMTT00000017225.1,XP_029769102.1,ENSTMTP00000016631.1
3402973,2587831,115271400,ENSTMTG00000007221,XM_029913853.1,ENSTMTT00000010245.1,XP_029769713.1,ENSTMTP00000009912.1
3402974,2587831,115271535,ENSTMTG00000012454,XM_029914354.1,ENSTMTT00000017564.1,XP_029770214.1,ENSTMTP00000016963.1
3402975,2587831,115271563,ENSTMTG00000012937,XM_029914401.1,ENSTMTT00000018218.1,XP_029770261.1,ENSTMTP00000017593.1


In [49]:
human = pd.read_table('Homo_sapiens.gene_info')


aliases = human['Synonyms'].tolist()
aliases_sep = []
for i in range(len(aliases)):
    my_list = aliases[i].split("|")
    my_list.append(human['Symbol'].iloc[i])
    aliases_sep.append(my_list)
human['Aliases'] = aliases_sep

human

,#tax_id,GeneID,Symbol,LocusTag,Synonyms,dbXrefs,chromosome,map_location,description,type_of_gene,Symbol_from_nomenclature_authority,Full_name_from_nomenclature_authority,Nomenclature_status,Other_designations,Modification_date,Feature_type,Aliases
0,9606,1,A1BG,-,A1B|ABG|GAB|HYST2477,MIM:138670|HGNC:HGNC:5|Ensembl:ENSG00000121410,19,19q13.43,alpha-1-B glycoprotein,protein-coding,A1BG,alpha-1-B glycoprotein,O,alpha-1B-glycoprotein|HEL-S-163pA|epididymis s...,20200313,-,"[A1B, ABG, GAB, HYST2477, A1BG]"
1,9606,2,A2M,-,A2MD|CPAMD5|FWP007|S863-7,MIM:103950|HGNC:HGNC:7|Ensembl:ENSG00000175899,12,12p13.31,alpha-2-macroglobulin,protein-coding,A2M,alpha-2-macroglobulin,O,alpha-2-macroglobulin|C3 and PZP-like alpha-2-...,20200510,-,"[A2MD, CPAMD5, FWP007, S863-7, A2M]"
2,9606,3,A2MP1,-,A2MP,HGNC:HGNC:8|Ensembl:ENSG00000256069,12,12p13.31,alpha-2-macroglobulin pseudogene 1,pseudo,A2MP1,alpha-2-macroglobulin pseudogene 1,O,pregnancy-zone protein pseudogene,20200313,-,"[A2MP, A2MP1]"
3,9606,9,NAT1,-,AAC1|MNAT|NAT-1|NATI,MIM:108345|HGNC:HGNC:7645|Ensembl:ENSG00000171428,8,8p22,N-acetyltransferase 1,protein-coding,NAT1,N-acetyltransferase 1,O,arylamine N-acetyltransferase 1|N-acetyltransf...,20200313,-,"[AAC1, MNAT, NAT-1, NATI, NAT1]"
4,9606,10,NAT2,-,AAC2|NAT-2|PNAT,MIM:612182|HGNC:HGNC:7646|Ensembl:ENSG00000156006,8,8p22,N-acetyltransferase 2,protein-coding,NAT2,N-acetyltransferase 2,O,arylamine N-acetyltransferase 2|N-acetyltransf...,20200405,-,"[AAC2, NAT-2, PNAT, NAT2]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61625,741158,8923215,trnD,-,-,-,MT,-,-,tRNA,-,-,-,-,20180406,-,"[-, trnD]"
61626,741158,8923216,trnP,-,-,-,MT,-,-,tRNA,-,-,-,-,20180406,-,"[-, trnP]"
61627,741158,8923217,trnA,-,-,-,MT,-,-,tRNA,-,-,-,-,20180406,-,"[-, trnA]"
61628,741158,8923218,COX1,-,-,-,MT,-,cytochrome c oxidase subunit I,protein-coding,-,-,-,cytochrome c oxidase subunit I,20180406,-,"[-, COX1]"


In [25]:
mart = pd.read_csv('mart_export.txt',delimiter = ',')
mart

,Gene stable ID,Gene stable ID version,Transcript stable ID,Transcript stable ID version,Gene name,HGNC symbol,Gene start (bp),Gene end (bp),Transcript start (bp),Transcript end (bp)
0,ENSG00000210049,ENSG00000210049.1,ENST00000387314,ENST00000387314.1,MT-TF,MT-TF,577,647,577,647
1,ENSG00000211459,ENSG00000211459.2,ENST00000389680,ENST00000389680.2,MT-RNR1,MT-RNR1,648,1601,648,1601
2,ENSG00000210077,ENSG00000210077.1,ENST00000387342,ENST00000387342.1,MT-TV,MT-TV,1602,1670,1602,1670
3,ENSG00000210082,ENSG00000210082.2,ENST00000387347,ENST00000387347.2,MT-RNR2,MT-RNR2,1671,3229,1671,3229
4,ENSG00000209082,ENSG00000209082.1,ENST00000386347,ENST00000386347.1,MT-TL1,MT-TL1,3230,3304,3230,3304
...,...,...,...,...,...,...,...,...,...,...
249784,ENSG00000284869,ENSG00000284869.2,ENST00000644579,ENST00000644579.2,EEFSEC,EEFSEC,128160388,128415576,128160388,128415576
249785,ENSG00000284869,ENSG00000284869.2,ENST00000644955,ENST00000644955.1,EEFSEC,EEFSEC,128160388,128415576,128160408,128415572
249786,ENSG00000284869,ENSG00000284869.2,ENST00000645384,ENST00000645384.1,EEFSEC,EEFSEC,128160388,128415576,128160575,128348438
249787,ENSG00000284869,ENSG00000284869.2,ENST00000646013,ENST00000646013.1,EEFSEC,EEFSEC,128160388,128415576,128343834,128348718


In [26]:
neighbors = pd.read_table('gene_neighbors')
neighbors

C:\Users\Brendan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,#tax_id,GeneID,genomic_accession.version,genomic_gi,start_position,end_position,orientation,chromosome,GeneIDs_on_left,distance_to_left,GeneIDs_on_right,distance_to_right,overlapping_GeneIDs,assembly
0,9770,807607,NC_001321.1,5819095,8388,9068,+,MT,807612|807613,234,807610,854,807608|807611,-
1,9770,807608,NC_001321.1,5819095,9068,9853,+,MT,807611|807612,650,807610|807616|807617,69,807607,-
2,9770,807609,NC_001321.1,5819095,4354,5397,+,MT,807618,209,807612|807613,384,-,-
3,9770,807610,NC_001321.1,5819095,9922,10267,+,MT,807607|807608|807611,69,807614|807615|807616|807617,71,-,-
4,9770,807611,NC_001321.1,5819095,8227,8418,+,MT,807612|807613,73,807608|807610,650,807607,-
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27241512,1710540,55900172,NZ_CP058335.1,1867135660,1203619,1204218,-,-,55900170|55900171,43,55900173|55900174,120,-,-
27241513,1710540,55900173,NZ_CP058335.1,1867135660,1204338,1205720,+,-,55900171|55900172,120,55900174|55900175,4,-,-
27241514,1710540,55900174,NZ_CP058335.1,1867135660,1205724,1206146,-,-,55900172|55900173,4,55900175|55900176,100,-,-
27241515,1710540,55900175,NZ_CP058335.1,1867135660,1206246,1207034,-,-,55900173|55900174,100,55900176|55900177,247,-,-


In [27]:
orthologs = pd.read_table('gene_orthologs')
orthologs

,#tax_id,GeneID,relationship,Other_tax_id,Other_GeneID
0,7955,30037,Ortholog,7897,102366554
1,7955,30037,Ortholog,7994,103038481
2,7955,30037,Ortholog,7998,108255935
3,7955,30037,Ortholog,8005,113577924
4,7955,30037,Ortholog,8010,105014395
...,...,...,...,...,...
4771230,10090,113939900,Ortholog,9913,113939901
4771231,10090,113939900,Ortholog,10116,113939903
4771232,10090,113939900,Ortholog,27679,113939905
4771233,10090,113939900,Ortholog,28377,113939909


In [28]:
j_entrez = pd.read_table('J_Entrez')
j_entrez

,--------------------------------------------------------
0,JrId: 1
1,JournalTitle: AADE editors' journal
2,MedAbbr: AADE Ed J
3,ISSN (Print): 0160-6999
4,ISSN (Online):
...,...
294986,MedAbbr: Acta Anal
294987,ISSN (Print): 0353-5150
294988,ISSN (Online): 1874-6349
294989,IsoAbbr: Acta Anal


In [29]:
def compare_sets(list_data):
    for ele in list_data:
        col_names = ele.columns
        print(col_names)
data_list = refseq,pubmed,ensembl,human,mart,neighbors,orthologs,j_entrez
compare_sets(data_list)

Index(['#tax_id', 'GeneID', 'status', 'RNA_nucleotide_accession.version',
       'RNA_nucleotide_gi', 'protein_accession.version', 'protein_gi',
       'genomic_nucleotide_accession.version', 'genomic_nucleotide_gi',
       'start_position_on_the_genomic_accession',
       'end_position_on_the_genomic_accession', 'orientation', 'assembly',
       'mature_peptide_accession.version', 'mature_peptide_gi', 'Symbol'],
      dtype='object')
Index(['#tax_id', 'GeneID', 'PubMed_ID'], dtype='object')
Index(['#tax_id', 'GeneID', 'Ensembl_gene_identifier',
       'RNA_nucleotide_accession.version', 'Ensembl_rna_identifier',
       'protein_accession.version', 'Ensembl_protein_identifier'],
      dtype='object')
Index(['#tax_id', 'GeneID', 'Symbol', 'LocusTag', 'Synonyms', 'dbXrefs',
       'chromosome', 'map_location', 'description', 'type_of_gene',
       'Symbol_from_nomenclature_authority',
       'Full_name_from_nomenclature_authority', 'Nomenclature_status',
       'Other_designations', 'Mod

In [106]:
def match_lists(data_list):
    for i in range(len(data_list)):
        for j in range(len(data_list)):
            if (i != j and i < j):
                ### translation 1 test col name
                test_col1 = data_list[i][1]

                ### translation 1 reference dataset (human,gene,mart)
                ref_df1 = pd.read_table(data_list[j][0])

                ### translation 1 reference col name
                ref_col1 = data_list[j][1]

                ### translation 1 output col name
                out_col1 = 'MATCH'
                
                trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]

                match_percent = combined_translate(data_list[i][0],'other', trans_list)
                print(match_percent," percent mismatch, ", data_list[i][0], 'to', data_list[j][0])


In [108]:
t2d = pd.read_table('t2d_SNP.txt')

# t2dm - https://www.genome.jp/kegg-bin/show_pathway?hsa04930
t2d

t2dm = pd.read_table('t2dm.txt')
t2dm
# https://www.genome.jp/kegg-bin/show_pathway?hsa04930

da1 = pd.read_table('diabetesassay1')
# https://www.nature.com/articles/s41467-018-06249-3
da1

ass_st = pd.read_table('association_study.txt')
t2dm
# https://onlinelibrary.wiley.com/doi/full/10.1111/j.1365-2362.2010.02454.x

novel_t2d = pd.read_table('novel_t2d.txt')
#https://journals.plos.org/plosgenetics/article?id=10.1371/journal.pgen.1003046

l1 = ['t2d_SNP.txt', 'Gene symbol']
l2 = ['t2dm.txt', 'Gene_name']
l3 = ['diabetesassay1','Human ortholog']
l4 = ['association_study.txt','Gene loci']
l5 = ['novel_t2d.txt','Internode gene']
match_lists([l1,l2,l3,l4,l5])

96.6667  percent mismatch,  t2d_SNP.txt to t2dm.txt
100.0  percent mismatch,  t2d_SNP.txt to diabetesassay1
80.0  percent mismatch,  t2d_SNP.txt to association_study.txt
100.0  percent mismatch,  t2d_SNP.txt to novel_t2d.txt
100.0  percent mismatch,  t2dm.txt to diabetesassay1
83.3333  percent mismatch,  t2dm.txt to association_study.txt
100.0  percent mismatch,  t2dm.txt to novel_t2d.txt
35.7143  percent mismatch,  diabetesassay1 to association_study.txt
100.0  percent mismatch,  diabetesassay1 to novel_t2d.txt
100.0  percent mismatch,  association_study.txt to novel_t2d.txt


In [105]:
def combined_translate(input_file, file_type, trans_list):
    if (file_type == 'csv'):
        df = pd.read_csv(input_file)
    if (file_type == 'excel'):
        df = pd.read_excel(input_file)
    if (file_type == 'other'):
        df = pd.read_table(input_file)
    
    for item in trans_list:
        test_col = item[0]
        reference_df = item[1]
        reference_col = item[2]
        output_col = item[3]
        
        
        start_param = df[test_col].tolist()
        in_list = reference_df[reference_col].tolist()
        out_listoflist = []

        for i in range(len(start_param)):
            param = start_param[i]

            match_list=[]
            for j in range(len(in_list)):
                if str(param) in str(in_list[j]):
                    index_match = reference_df.iloc[j]
                    match_list.append(index_match)

            out_list = []
            for k in range(len(match_list)):
                if (output_col == 'MATCH'):
                    out_list.append(match_list[k][reference_col])
                else: 
                    out_list.append(match_list[k][output_col])

            out_listoflist.append(out_list)
        out_colname = 'output ' + reference_col
        df[out_colname] = out_listoflist

    
        nonmatches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)==0 or x != x or x=='-']
        matches_ind = [ind for ind, x in enumerate(df[out_colname]) if len(x)!=0 or x == x or x!='-']
        nonmatches = df.iloc[nonmatches_ind]
        matches = df.iloc[matches_ind]
        
        return round(100*len(nonmatches)/len(df),4)
#         print(round(100*len(nonmatches)/len(df),4)," percent mismatch, ",input_file, 'to')
    
#     print(df)
    
        
          

In [68]:
### Name of input data file
input_file = 'association_study.txt'

### Type of file (excel, csv, or other)
file_type = 'other'

### translation 1 test col name
test_col1 = 'Gene loci'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'MATCH'

### translation 1 use only if one translation, otherwise delete
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
combined_translate(input_file,file_type,trans_list)

5.6604  percent mismatch,  Gene loci
        Gene loci                                     output Aliases
0           PPARG  [[CIMT1, GLM1, NR1C3, PPARG1, PPARG2, PPARG5, ...
1          KCNJ11  [[BIR, HHF2, IKATP, KIR6.2, MODY13, PHHI, PNDM...
2          TCF7L2                            [[TCF-4, TCF4, TCF7L2]]
3            WFS1                 [[CTRCT41, WFRS, WFS, WFSL, WFS1]]
4           HNF1B  [[FJHN, HNF-1-beta, HNF-1B, HNF1beta, HNF2, HP...
5             FTO  [[ALKBH9, BMIQ14, GDFD, FTO], [-, FTO-IT1], [-...
6         SLC30A8                           [[ZNT8, ZnT-8, SLC30A8]]
7            HHEX         [[HEX, HMPH, HOX11L-PEN, PRH, PRHX, HHEX]]
8          CDKAL1                                      [[-, CDKAL1]]
9         IGF2BP2  [[IMP-2, IMP2, VICKZ2, IGF2BP2], [C3orf65, IGF...
10         CDKN2A  [[ARF, CDK4I, CDKN2, CMM2, INK4, INK4A, MLM, M...
11         TSPAN8                         [[CO-029, TM4SF3, TSPAN8]]
12        ADAMTS9  [[-, ADAMTS9], [-, ADAMTS9-AS2], [-, ADAMTS9-A.

In [50]:
### Name of input data file
input_file = 't2d_SNP.txt'

### Type of file (excel, csv, or other)
file_type = 'other'

### translation 1 test col name
test_col1 = 'Gene symbol'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'GeneID'

### translation 1 use only if one translation, otherwise delete
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
combined_translate(input_file,file_type,trans_list)

0.0  percent mismatch,  Gene symbol
            SNP Gene symbol  Chromosome Position (bp)*  \
0     rs4740283     RAPGEF1           9    131,477,850   
1    rs2021966†       ENPP1           6    132,192,132   
2   rs1042522†‡        TP53          17      7,520,197   
3     rs1882095        NRF1           7    128,991,595   
4    rs10513684      SLC2A2           3    172,206,912   
5     rs1801282       PPARG           3     12,368,125   
6      rs222852      SLC2A4          17      7,081,330   
7     rs4843165       FOXC2          16     85,162,542   
8       rs5400‡      SLC2A2           3    172,215,002   
9      rs858341       ENPP1           6    132,202,148   
10    rs1349498     RAPGEF4           2    173,418,113   
11    rs8069976       SOCS3          17     73,861,445   
12    rs3769249     RAPGEF4           2    173,648,169   
13   rs17280845        CBLB           3    106,927,226   
14      rs5219‡      KCNJ11          11     17,366,148   
15   rs10915239        MECR         

In [54]:
### Name of input data file
input_file = 'T2dm.txt'

### Type of file (excel, csv, or other)
file_type = 'other'

### translation 1 test col name
test_col1 = 'Gene_name'

### translation 1 reference dataset (human,gene,mart)
ref_df1 = human

### translation 1 reference col name
ref_col1 = 'Aliases'
    
### translation 1 output col name
out_col1 = 'GeneID'

### translation 1 use only if one translation, otherwise delete
trans_list = [[test_col1,ref_df1,ref_col1,out_col1]]
combined_translate(input_file,file_type,trans_list)

12.5  percent mismatch,  Gene_name
   Gene_name                                     output Aliases
0      ADIPO    [9370, 51094, 79602, 390503, 645220, 100874095]
1      GLUT4                                             [6517]
2        INS  [2305, 3342, 3416, 3630, 3638, 3640, 3641, 364...
3       INSR                                       [3643, 3645]
4       IRS1                      [3667, 53833, 112802, 147183]
5        IRS  [2618, 3376, 3667, 8471, 8660, 10458, 53833, 5...
6       P13K                                                 []
7       SOCS  [1154, 8651, 8835, 9021, 9306, 9655, 30837, 12...
8        ERK  [1942, 1943, 1944, 1945, 1946, 1947, 1948, 194...
9        IKK           [1147, 3551, 8517, 9641, 117166, 124857]
10       JNK              [5599, 5601, 5602, 5609, 6416, 23005]
11       TNF  [355, 356, 608, 690, 939, 943, 944, 958, 959, ...
12      TNFA  [1942, 5806, 7124, 7126, 7127, 7128, 7130, 713...
13      mTOR                                                 []
14   

In [ ]:
l1 = []